In [50]:
import constti
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import numpy as np
from pathlib import Path

team_number = 20
url1 = "https://fantasy.premierleague.com/api/bootstrap-static/"
url2 = "https://fantasy.premierleague.com/api/entry/698498/history/"
url3 = "https://fantasy.premierleague.com/api/event/6/live/"
url4 = "https://fantasy.premierleague.com/api/fixtures"

#NaNs to zeros
def toint(a):
    if np.isnan(a):
        return 0
    else: return int(a)

#If no matches played not to devide by zero
def noZ(a):
    b = a.copy()
    for i in range(len(b)):
        if b[i] == 0:
            b[i]=1
    return b


p1 = requests.get(url1)
#page1 = BeautifulSoup(p1.text)
#data1 = str(page1.p)[3:-4]
data1 = p1.text

d1 = json.loads(data1)
bigTable = pd.DataFrame(d1['elements'])
bigTable = bigTable[['team', 'element_type', 'web_name', 'goals_scored', 'assists', 'bonus', 'event_points', 'total_points', 
               'saves', 'own_goals', 'clean_sheets', 'penalties_missed', 'penalties_saved', 'yellow_cards', 'red_cards', 
               'minutes', 'bps', 'creativity', 'threat', 'ict_index', 'influence',
               'value_season', 'form', 'value_form', 'points_per_game', 
               'goals_conceded', 
               'in_dreamteam', 'dreamteam_count',
               'now_cost', 'cost_change_event', 'cost_change_event_fall',
               'cost_change_start', 'cost_change_start_fall', 'selected_by_percent',
               'transfers_in_event', 'transfers_out_event', 'transfers_in', 'transfers_out', 
               'chance_of_playing_this_round', 'chance_of_playing_next_round', 'news_added', 'news', 'status', 
               'ep_this', 'ep_next', 'first_name', 'second_name', 'team_code', 'id', 'photo', 'special', 'squad_number', 'code']]
bigTable['full_name'] = bigTable['first_name'] + ' ' + bigTable['second_name']
bigTable.to_csv(Path('in/fpltable.csv'))

p4 = requests.get(url4)
#page4 = BeautifulSoup(p4.text)
#data4 = str(page4.p)[3:-4]
d4 = json.loads(p4.text)
Fixtures = pd.DataFrame(d4)
Fixtures.to_csv(Path('in/fplfixtures.csv'))

#Figuring out gameweek of the last ended match - lastGW

#if Fixtures.at[1,'minutes'] != 90:
#    lastGW = 0
#elif Fixtures.at[len(Fixtures)-1,'minutes'] == 90:
#    lastGW = Fixtures.at[len(Fixtures)-1,'event']
#else:
#    for i in range(1,len(Fixtures)):
#        if Fixtures.at[i,'minutes'] != 90:
#            lastGW = Fixtures.at[i-1,'event']
#            break
#lastGW = int(lastGW)

firstr = len(Fixtures)+1
lastr = 0
for i in range(len(Fixtures)):
    if Fixtures.at[i,'finished']==True:
        firstr = min(firstr, i)
        lastr = i
if firstr < len(Fixtures)+1:
    lastGW = int(Fixtures.at[lastr-1,'event'])
else: lastGW = 0
if lastr == len(Fixtures):
    lastGW = int(Fixtures.at[lastr,'event'])







Gameweeks = pd.DataFrame()
for i in range(1,2*team_number - 1):
    url = "https://fantasy.premierleague.com/api/event/" + str(i) + "/live/"
    p = requests.get(url)
    #page = BeautifulSoup(p.text)
    #data = str(page.p)[3:-4]
    d = json.loads(p.text)
    nexTour = pd.DataFrame(d['elements'])
    
    if not nexTour.empty:
        nt1 = pd.DataFrame(nexTour['stats'].tolist())
        nt1['id'] = nexTour['id']
        nt1['gameweek'] = i
        nt1['fixture'] = [nexTour.at[i,'explain'][0]['fixture'] if not nexTour.at[i,'explain']==[] \
                          else '' for i in nexTour.index]
        #nt1.index = nt1['gameweek']*1000+nt1['id']
        Gameweeks = Gameweeks.append(nt1, ignore_index=True)
        print(i)

 

teams = dict(zip(pd.DataFrame(d1['teams'])['id'],pd.DataFrame(d1['teams'])['name']))
players = dict(zip(bigTable['id'],bigTable['full_name']))
teamplayers = dict(zip(bigTable['id'],bigTable['team']))

Gameweeks['team'] = [teamplayers[i] for i in Gameweeks['id']]

Gameweeks['threat'] = pd.to_numeric(Gameweeks['threat'])
Gameweeks['creativity'] = pd.to_numeric(Gameweeks['creativity'])

Gameweeks['team_a'] = [Fixtures[(Fixtures['event'] == Gameweeks.at[i,'gameweek']) & \
                                         ((Fixtures['team_a'] == Gameweeks.at[i,'team'])|\
                                          (Fixtures['team_h'] == Gameweeks.at[i,'team']))]['team_a'] \
                            for i in Gameweeks.index]


#Kill postponed matches lines

li =[]
for i in Gameweeks.index:
    if Gameweeks.at[i,'team_a'].empty:
        li.append(i)
Gameweeks = Gameweeks.drop(li)
        

Gameweeks['team_a'] = [int(Gameweeks.at[i,'team_a']) for i in Gameweeks.index]

Gameweeks['team_h'] = [int(Fixtures[(Fixtures['event'] == Gameweeks.at[i,'gameweek']) & \
                                         ((Fixtures['team_a'] == Gameweeks.at[i,'team'])|\
                                          (Fixtures['team_h'] == Gameweeks.at[i,'team']))]['team_h']) \
                            for i in Gameweeks.index]

Gameweeks['teamAgainst'] = [Gameweeks.at[i,'team_a'] if Gameweeks.at[i,'team'] == Gameweeks.at[i,'team_h']\
                            else Gameweeks.at[i,'team_h'] \
                           for i in Gameweeks.index]
Gameweeks['side'] = ['home' if Gameweeks.at[i,'team'] == Gameweeks.at[i,'team_h']\
                            else 'away' \
                           for i in Gameweeks.index]

del Gameweeks['team_a']
del Gameweeks['team_h']


Gameweeks.to_csv(Path('in/fplgameweeks.csv'))
Gameweeks

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,...,threat,ict_index,total_points,in_dreamteam,id,gameweek,fixture,team,teamAgainst,side
0,90,0,0,1,0,0,0,0,0,0,...,0.0,1.8,7,False,14,1,10,1,13,away
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,False,27,1,6,2,17,away
2,90,0,0,1,0,0,0,0,0,0,...,0.0,2.5,9,False,47,1,7,4,18,away
3,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,False,48,1,7,4,18,away
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,False,72,1,2,3,15,home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10507,58,0,0,0,4,0,0,0,0,0,...,6.0,0.7,1,False,554,19,187,13,12,away
10508,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,False,559,19,190,20,11,home
10509,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,False,561,19,185,8,5,home
10510,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,False,570,19,185,8,5,home


In [ ]:
#Matches
TeamMatches = pd.DataFrame()
TeamMatches['Matches'] = [len(Fixtures[Fixtures['finished']&((Fixtures['team_a']==i)|(Fixtures['team_h']==i))]) \
                              for i in range(1,team_number+1)]

PlayerMatches = pd.DataFrame()
PlayerMatches['id'] = bigTable['id']
PlayerMatches['Team number'] = [bigTable[bigTable['id'] == i]['team'].sum() for i in players.keys()]
PlayerMatches['Team'] = [teams[PlayerMatches.at[i,'Team number']] for i in range(len(players))]
PlayerMatches['Team games'] = [TeamMatches.at[PlayerMatches.at[i,'Team number']-1,'Matches'] for i in PlayerMatches.index]
PlayerMatches['Played'] = [len(Gameweeks[(Gameweeks['id']==i)&(Gameweeks['minutes']>0)]) \
                        for i in PlayerMatches['id']]


#Team tables

print(0)
#1. Creating  a table with average threat and GW threats for teams

TeamThreat = pd.DataFrame()
TeamThreat['id'] = pd.DataFrame(d1['teams'])['id']
TeamThreat['Team'] = pd.DataFrame(d1['teams'])['name']
TeamThreat['Threat av'] = np.zeros(len(TeamThreat))
TeamThreat['Matches'] = TeamMatches['Matches']

for j in range(lastGW,0,-1):
    TeamThreat['Threat GW'+str(j)] = [Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==j)]['threat'].sum() \
                                      for i in range(1,team_number+1)]
    

TeamThreat['Threat av'] = [Gameweeks[Gameweeks['team']==i]['threat'].sum() for i in range(1,team_number+1)] \
    /noZ(TeamMatches['Matches'])
print(1)
#2. Creating  a table with average creativity and GW creativities for teams

TeamCreativity = pd.DataFrame()
TeamCreativity['id'] = pd.DataFrame(d1['teams'])['id']
TeamCreativity['Team'] = pd.DataFrame(d1['teams'])['name']
TeamCreativity['Creativity av'] = np.zeros(len(TeamCreativity))
TeamCreativity['Matches'] = TeamMatches['Matches']

for j in range(lastGW,0,-1):
    TeamCreativity['Creativity GW'+str(j)] = [Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==j)] \
            ['creativity'].sum() for i in range(1,team_number+1)]
    

TeamCreativity['Creativity av'] = [Gameweeks[Gameweeks['team']==i]['creativity'].sum() for i in range(1,team_number+1)] \
        /noZ(TeamMatches['Matches'])
print(2)
#3. Creating  a table with average threat allowed by teams and GW threat allowed

TableDefence = pd.DataFrame()
TableDefence['id'] = pd.DataFrame(d1['teams'])['id']
TableDefence['Team'] = pd.DataFrame(d1['teams'])['name']
TableDefence['Threat allowed av'] = np.zeros(len(TableDefence))
TableDefence['Matches'] = TeamMatches['Matches']

for j in range(lastGW,0,-1):
    TableDefence['Threat allowed GW'+str(j)] = [Gameweeks[(Gameweeks['teamAgainst']==i)&(Gameweeks['gameweek']==j)] \
            ['threat'].sum() for i in range(1,team_number+1)]

TableDefence['Threat allowed av'] = [Gameweeks[Gameweeks['teamAgainst']==i]['threat'].sum() for i in range(1,team_number+1)] \
        /noZ(TeamMatches['Matches'])

threatAllowedAv = TableDefence['Threat allowed av'].mean()
print(3)
#4. Creating  a table with average adjusted threat and GW threats adj for teams

TeamThreatAd = pd.DataFrame()
TeamThreatAd['id'] = pd.DataFrame(d1['teams'])['id']
TeamThreatAd['Team'] = pd.DataFrame(d1['teams'])['name']
TeamThreatAd['Threat av adj'] = np.zeros(len(TeamThreatAd))
TeamThreatAd['Matches'] = TeamMatches['Matches']

for j in range(lastGW,0,-1):    
    TeamThreatAd['Threat GW'+str(j)+' adj'] = [TeamThreat.at[i-1,'Threat GW'+str(j)]*threatAllowedAv/ \
                TableDefence.at[toint(Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==j)]['teamAgainst'].mean()-1), \
                'Threat allowed av'] \
                                             for i in range(1,team_number+1)]
    TeamThreatAd['Threat av adj'] = TeamThreatAd['Threat av adj']  + TeamThreatAd['Threat GW'+str(j)+' adj']

TeamThreatAd['Threat av adj'] = TeamThreatAd['Threat av adj']/noZ(TeamMatches['Matches'])
print(4)
#5. Creating  a table with average adjusted creativity and GW creativities adj for teams

TeamCreativityAd = pd.DataFrame()
TeamCreativityAd['id'] = pd.DataFrame(d1['teams'])['id']
TeamCreativityAd['Team'] = pd.DataFrame(d1['teams'])['name']
TeamCreativityAd['Creativity av adj'] = np.zeros(len(TeamCreativityAd))
TeamCreativityAd['Matches'] = TeamMatches['Matches']

for j in range(lastGW,0,-1):    
    TeamCreativityAd['Creativity GW'+str(j)+' adj'] = [TeamCreativity.at[i-1,'Creativity GW'+str(j)]*threatAllowedAv/ \
                TableDefence.at[toint(Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==j)]['teamAgainst'].mean()-1), \
                'Threat allowed av'] \
                                             for i in range(1,team_number+1)]
    TeamCreativityAd['Creativity av adj'] = TeamCreativityAd['Creativity av adj']  + \
        TeamCreativityAd['Creativity GW'+str(j)+' adj']

TeamCreativityAd['Creativity av adj'] = TeamCreativityAd['Creativity av adj']/noZ(TeamMatches['Matches'])
print(5)
#6. Creating  a table with average threat allowed adjusted by teams and GW threat allowed adjusted

TableDefenceAd = pd.DataFrame()
TableDefenceAd['id'] = pd.DataFrame(d1['teams'])['id']
TableDefenceAd['Team'] = pd.DataFrame(d1['teams'])['name']
TableDefenceAd['Threat allowed av adj'] = np.zeros(len(TableDefenceAd))
TableDefenceAd['Matches'] = TeamMatches['Matches']

for j in range(lastGW,0,-1):    
    TableDefenceAd['Threat allowed GW'+str(j)+' adj'] = [TableDefence.at[i-1,'Threat allowed GW'+str(j)]*threatAllowedAv/ \
                TeamThreat.at[toint(Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==j)]['teamAgainst'].mean()-1), \
                'Threat av'] \
                                             for i in range(1,team_number+1)]
    TableDefenceAd['Threat allowed av adj'] = TableDefenceAd['Threat allowed av adj']  + \
        TableDefenceAd['Threat allowed GW'+str(j)+' adj']

TableDefenceAd['Threat allowed av adj'] = TableDefenceAd['Threat allowed av adj']/noZ(TeamMatches['Matches'])

print(6)


#Total Team Table

TableTeams = pd.DataFrame()
TableTeams['id'] = pd.DataFrame(d1['teams'])['id']
TableTeams['Team'] = pd.DataFrame(d1['teams'])['name']

TableTeams['Threat adjusted'] = TeamThreatAd['Threat av adj']
TableTeams['Threat'] = TeamThreat['Threat av']
TableTeams['Creativity adjusted'] = TeamCreativityAd['Creativity av adj']
TableTeams['Creativity'] = TeamCreativity['Creativity av']
TableTeams['Threat allowed adjusted'] = TableDefenceAd['Threat allowed av adj']
TableTeams['Threat allowed'] = TableDefence['Threat allowed av']
print(7)



#PLayer Tables


#1 Players Threat

PlayerThreat = pd.DataFrame()
PlayerThreat['id'] = bigTable['id']
PlayerThreat['Name'] = bigTable['full_name']
PlayerThreat['Team'] = PlayerMatches['Team']
PlayerThreat['Threat per fixture'] = np.zeros(len(players))
PlayerThreat['Threat per game'] = np.zeros(len(players))

for j in range(lastGW,0,-1):
    PlayerThreat['Threat GW'+str(j)] = [Gameweeks[(Gameweeks['id']==PlayerThreat.iat[i,0])&(Gameweeks['gameweek']==j)] \
                                    ['threat'].sum() \
                                    for i in range(len(players))]
    PlayerThreat['Threat per game'] = PlayerThreat['Threat per game'] + PlayerThreat['Threat GW'+str(j)]

PlayerThreat['Threat per fixture'] = PlayerThreat['Threat per game']/noZ(PlayerMatches['Team games'])
PlayerThreat['Threat per game'] = PlayerThreat['Threat per game']/noZ(PlayerMatches['Played'])
print(9)
#2 Players Creativity

PlayerCreativity = pd.DataFrame()
PlayerCreativity['id'] = bigTable['id']
PlayerCreativity['Name'] = bigTable['full_name']
PlayerCreativity['Team'] = PlayerMatches['Team']
PlayerCreativity['Creativity per fixture'] = np.zeros(len(players))
PlayerCreativity['Creativity per game'] = np.zeros(len(players))

for j in range(lastGW,0,-1):
    PlayerCreativity['Creativity GW'+str(j)] = [Gameweeks[(Gameweeks['id']==PlayerThreat.iat[i,0])&\
                    (Gameweeks['gameweek']==j)]['creativity'].sum() for i in range(len(players))]
    PlayerCreativity['Creativity per game'] = PlayerCreativity['Creativity per game']\
        + PlayerCreativity['Creativity GW'+str(j)]

PlayerCreativity['Creativity per fixture'] = PlayerCreativity['Creativity per game']/noZ(PlayerMatches['Team games'])
PlayerCreativity['Creativity per game'] = PlayerCreativity['Creativity per game']/noZ(PlayerMatches['Played'])
print(10)
#3 Players Threat Adjusted

PlayerThreatAd = pd.DataFrame()
PlayerThreatAd['id'] = bigTable['id']
PlayerThreatAd['Name'] = bigTable['full_name']
PlayerThreatAd['Team number'] = PlayerMatches['Team number']
PlayerThreatAd['Team'] = PlayerMatches['Team']
PlayerThreatAd['Threat per fixture adj'] = np.zeros(len(players))
PlayerThreatAd['Threat per game adj'] = np.zeros(len(players))
for j in range(lastGW,0,-1):
        
    PlayerThreatAd['Threat GW'+str(j) + 'adj'] = [PlayerThreat.at[i,'Threat GW'+str(j)]*threatAllowedAv/ \
            TableDefence.at[toint(Gameweeks[(Gameweeks['team']==PlayerThreatAd.at[i,'Team number'])&\
            (Gameweeks['gameweek']==j)]['teamAgainst'].mean()-1), \
            'Threat allowed av']  for i in range(len(players))]                     
    
    PlayerThreatAd['Threat per game adj'] = PlayerThreatAd['Threat per game adj'] +\
            PlayerThreatAd['Threat GW'+str(j) + 'adj']
PlayerThreatAd['Threat per fixture adj'] = PlayerThreatAd['Threat per game adj']/noZ(PlayerMatches['Team games'])
PlayerThreatAd['Threat per game adj'] = PlayerThreatAd['Threat per game adj']/noZ(PlayerMatches['Played'])
print(11)

#4 PLayers Creativity Adjusted

PlayerCreativityAd = pd.DataFrame()
PlayerCreativityAd['id'] = bigTable['id']
PlayerCreativityAd['Name'] = bigTable['full_name']
PlayerCreativityAd['Team number'] = PlayerMatches['Team number']
PlayerCreativityAd['Team'] = PlayerMatches['Team']
PlayerCreativityAd['Creativity per fixture adj'] = np.zeros(len(players))
PlayerCreativityAd['Creativity per game adj'] = np.zeros(len(players))

for j in range(lastGW,0,-1):
        
    PlayerCreativityAd['Creativity GW'+str(j) + 'adj'] = [PlayerCreativity.at[i,'Creativity GW'+str(j)]*threatAllowedAv/ \
            TableDefence.at[toint(Gameweeks[(Gameweeks['team']==PlayerCreativityAd.at[i,'Team number'])&\
                                          (Gameweeks['gameweek']==j)]['teamAgainst'].mean()-1), \
            'Threat allowed av']  for i in range(len(players))]                     
    
    PlayerCreativityAd['Creativity per game adj'] = PlayerCreativityAd['Creativity per game adj'] +\
            PlayerCreativityAd['Creativity GW'+str(j) + 'adj']

PlayerCreativityAd['Creativity per fixture adj'] = PlayerCreativityAd['Creativity per game adj']/\
            noZ(PlayerMatches['Team games'])
PlayerCreativityAd['Creativity per game adj'] = PlayerCreativityAd['Creativity per game adj']/\
            noZ(PlayerMatches['Played'])
print(12)


#Tables2Files

del TeamThreat['id']
TeamThreat.sort_values('Threat av', ascending = False, inplace = True)
TeamThreat.index = np.arange(1, len(TeamThreat) + 1)
TeamThreat.to_csv(Path('out/TeamThreat.csv'))

del TeamCreativity['id']
TeamCreativity.sort_values('Creativity av', ascending = False, inplace = True)
TeamCreativity.index = np.arange(1, len(TeamCreativity) + 1)
TeamCreativity.to_csv(Path('out/TeamCreativity.csv'))

del TableDefence['id']
TableDefence.sort_values('Threat allowed av', ascending = True, inplace = True)
TableDefence.index = np.arange(1, len(TableDefence) + 1)
TableDefence.to_csv(Path('out/TableDefence.csv'))

del TeamThreatAd['id']
TeamThreatAd.sort_values('Threat av adj', ascending = False, inplace = True)
TeamThreatAd.index = np.arange(1, len(TeamThreatAd) + 1)
TeamThreatAd.to_csv(Path('out/TeamThreatAd.csv'))

del TeamCreativityAd['id']
TeamCreativityAd.sort_values('Creativity av adj', ascending = False, inplace = True)
TeamCreativityAd.index = np.arange(1, len(TeamCreativityAd) + 1)
TeamCreativityAd.to_csv(Path('out/TeamCreativityAd.csv'))

del TableDefenceAd['id']
TableDefenceAd.sort_values('Threat allowed av adj', ascending = True, inplace = True)
TableDefenceAd.index = np.arange(1, len(TableDefenceAd) + 1)
TableDefenceAd.to_csv(Path('out/TableDefenceAd.csv'))

del TableTeams['id']
TableTeams.sort_values('Threat adjusted', ascending = False, inplace = True)
TableTeams.index = np.arange(1, len(TableTeams) + 1)
TableTeams.to_csv(Path('out/TableTeams.csv'))

del PlayerThreat['id']
PlayerThreat.sort_values('Threat per fixture', ascending = False, inplace = True)
PlayerThreat.index = np.arange(1, len(players) + 1)
PlayerThreat.to_csv(Path('out/PlayerThreat.csv'))

del PlayerCreativity['id']
PlayerCreativity.sort_values('Creativity per fixture', ascending = False, inplace = True)
PlayerCreativity.index = np.arange(1, len(players) + 1)
PlayerCreativity.to_csv(Path('out/PlayerCreativity.csv'))

del PlayerThreatAd['id']
del PlayerThreatAd['Team number']
PlayerThreatAd.sort_values('Threat per fixture adj', ascending = False, inplace = True)
PlayerThreatAd.index = np.arange(1, len(players) + 1)
PlayerThreatAd.to_csv(Path('out/PlayerThreatAd.csv'))

del PlayerCreativityAd['id']
del PlayerCreativityAd['Team number']
PlayerCreativityAd.sort_values('Creativity per fixture adj', ascending = False, inplace = True)
PlayerCreativityAd.index = np.arange(1, len(players) + 1)
PlayerCreativityAd.to_csv(Path('out/PlayerCreativityAd.csv'))

PlayerThreatAd

0
1
2
3
4
5
6
7
9
10
11


In [48]:
len(Fixtures[(Fixtures['finished'])&(Fixtures['event']==19)&((Fixtures['team_a']==20) | (Fixtures['team_h']==20))])

0

In [38]:
b = {}
a = [Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==19)]['fixture'] \
                                      for i in range(1,team_number+1)]
TeamThreat['Threat GW19'] = [Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==19)]['threat'].sum() \
                                      for i in range(1,team_number+1)]
a

[9932     182
 9975     182
 10000    182
 10001    182
 10002    182
 10003    182
 10004    182
 10005    182
 10006    182
 10007    182
 10008    182
 10009    182
 10040    182
 10164    182
 10178    182
 10196    182
 10197    182
 10199    182
 10200    182
 10201    182
 10202    182
 10387    182
 10394    182
 10395    182
 10396    182
 10418    182
 10431    182
 10433    182
 10434    182
 10435    182
 10498    182
 Name: fixture, dtype: object, 9933     181
 9940     181
 9976     181
 9981     181
 10010    181
 10011    181
 10012    181
 10013    181
 10113    181
 10156    181
 10159    181
 10161    181
 10162    181
 10203    181
 10204    181
 10205    181
 10206    181
 10207    181
 10208    181
 10209    181
 10210    181
 10211    181
 10386    181
 10388    181
 10397    181
 10436    181
 10484    181
 Name: fixture, dtype: object, 9936     182
 9937     182
 9938     182
 9983     182
 10021    182
 10022    182
 10023    182
 10024    182
 10025    182
 1

In [32]:
set(Gameweeks['fixture'])

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 181,
 182,
 183,
 184,
 185,
 186

In [30]:
A.append(B, ignore_index=True)

,0,1,2
0,1,2,3
1,2,3,4
2,1,2,3
3,2,3,4


In [65]:
url = "https://fantasy.premierleague.com/api/event/18/live/"
p = requests.get(url)

d = json.loads(p.text)
nexTour = pd.DataFrame(d['elements'])
nt1 = pd.DataFrame(nexTour['stats'].tolist())
nt1['id'] = nexTour['id']
nt1['gameweek'] = i
nt1['fixture'] = [nexTour.at[i,'explain'] for i in nexTour.index]
nt1.to_csv(Path('in/nt1.csv'))
nt1



,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,...,bps,influence,creativity,threat,ict_index,total_points,in_dreamteam,id,gameweek,fixture
0,90,0,0,1,0,0,0,0,0,0,...,21,5.2,0.0,0.0,0.5,6,False,14,18,"[{'fixture': 174, 'stats': [{'identifier': 'mi..."
1,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0,False,27,18,"[{'fixture': 171, 'stats': [{'identifier': 'mi..."
2,90,0,0,0,1,0,0,0,0,0,...,14,14.2,0.0,0.0,1.4,2,False,47,18,"[{'fixture': 173, 'stats': [{'identifier': 'mi..."
3,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0,False,48,18,"[{'fixture': 173, 'stats': [{'identifier': 'mi..."
4,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0,False,72,18,"[{'fixture': 172, 'stats': [{'identifier': 'mi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,13,0,0,0,0,0,0,0,0,0,...,3,0.0,0.0,0.0,0.0,1,False,554,18,"[{'fixture': 176, 'stats': [{'identifier': 'mi..."
574,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0,False,559,18,"[{'fixture': 177, 'stats': [{'identifier': 'mi..."
575,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0,False,561,18,"[{'fixture': 174, 'stats': [{'identifier': 'mi..."
576,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0,False,570,18,"[{'fixture': 174, 'stats': [{'identifier': 'mi..."


In [46]:
Gameweeks.columns[21]

'team'